In [2]:
from lxml import etree as ET
from bokeh.io import show, output_file
from bokeh.plotting import figure
import regex as re



ModuleNotFoundError: No module named 'regex'

In [3]:
with open("textM1.xml") as f:
    xml = ET.parse(f)
    
counts = []
identifiers = []

manuscrit = "M1"
# Je récupère les @types et je les mets dans une liste de valeur unique (set) puis je les trie (sorted)
cats = sorted(list(set([str(x) for x in xml.xpath("//@type")])))

# Passage pour compter les mots
_punc = re.compile("([^<>/\w\s])")
_spaces = re.compile("([\s\u00a0]+)")

def count_words(text):
    text = _spaces.sub(" ", _punc.sub(" \1 ", text))
    return len(text.split()), text
# Fin des définitions d'outils pour compter les mots

# Pour chaque chapitre et chaque "pas" de chapitre (0, chapitre1), (1, chapitre2)
for chap_no, chap in enumerate(xml.xpath("./div")):
    chap_no += 1
    for sect_no, section in enumerate(chap.xpath("./div")):
        sect_no += 1
        for p_no, p in enumerate(section.xpath("./p")):
            p_no += 1
            # Création d'une chaine identifiant nb_chap.nb_sect.nb_p
            identifier = "{}.{}.{}".format(chap_no, sect_no, p_no)
            # J'ajoute aussi un numero sequentiel de paragraph: les paragraphes sont numérotés de 1 à la fin
            identifiers.append(len(identifiers) + 1)
            # Création du dictionnaire où il y aura les stats1
            stats = {
                "identifier": identifier,
                "neutral": 0,
                "edition": 0,
                manuscrit: 0,
                **{cat: 0 for cat in cats}
            }
            
            # Récupération du texte non-spécifique
            for text_node in p.xpath("./text()"):
                cnt, txt = count_words(str(text_node).strip())
                if cnt:
                    stats["neutral"] += cnt
            
            # Récupération du texte spécifique à l'Édition (et donc en contradiction avec autre manuscrit)
            for text_node in p.xpath("./choix/Edition/text()"):
                cnt, txt = count_words(str(text_node).strip())
                if cnt:
                    stats["edition"] += cnt
            
            # Pour calcul total, addition du spécifique à l'édition et du général
            stats["edition_real"] = stats["edition"] + stats["neutral"]
            
            # Et maintenant, par catégorie de type
            for manuscrit_var in p.xpath("./choix/{}".format(manuscrit)):
                cnt, txt = count_words(str(manuscrit_var.text).strip())
                if cnt:
                    # Je rajoute aussi en fonction de @type
                    stats[manuscrit_var.attrib["type"]] += cnt
                    # On ajoute aussi au général
                    stats[manuscrit] += cnt
            
            stats["diff"] = stats["edition"] - stats[manuscrit]
            counts.append(stats)

NameError: name 're' is not defined